In [2]:
import numpy as np
import os
import time
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D, Dense, Dropout,Activation,Flatten
import tensorflow as tf

from keras.applications.vgg16 import preprocess_input
from keras.layers import Input
from keras.models import Model
import keras.utils.np_utils as np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

In [3]:
PATH = os.getcwd()
# Define data path
data_path = PATH + '/archive'
data_dir_list = ['Positive','Negative']

img_data_list=[]

In [4]:
from tqdm import tqdm

In [5]:
for dataset in tqdm(data_dir_list):
	img_list=os.listdir(data_path+"/"+ dataset)
	#print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
	for img in tqdm(img_list):
		img_path = data_path + '/'+ dataset + '/'+ img 
		imge = image.load_img(img_path, target_size=(224, 224))
		x = image.img_to_array(imge)
		x = np.expand_dims(x, axis=0)
		x = preprocess_input(x)
		#print('Input image shape:', x.shape)
		img_data_list.append(x)


 62%|██████▏   | 12456/20000 [00:14<00:08, 880.09it/s]


 25%|██▌       | 5059/20000 [00:05<00:16, 900.63it/s]


 90%|████████▉ | 17976/20000 [00:20<00:02, 906.39it/s]


100%|██████████| 2/2 [00:45<00:00, 22.57s/it]


In [6]:
img_data = np.array(img_data_list)
#img_data = img_data.astype('float32')
#print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
#print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)

(40000, 224, 224, 3)


In [7]:
num_classes = 2
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:int(num_of_samples/2)]=0

labels[3*int(num_of_samples/2):num_of_samples]=1


names = ['Positive','Negative']
# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

In [8]:
#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=2)


In [9]:
image_input = Input(shape=(224,224, 3))


In [10]:
model = tf.keras.applications.ResNet50(input_tensor=image_input, include_top=True,weights='imagenet')
#model.summary()
x= Flatten(name='flatten')(model.layers[-1].output)
out = Dense(num_classes, activation='softmax', name='output_layer')(x)
custom_resnet_model = Model(inputs=image_input,outputs= out)
#custom_resnet_model.summary()

In [11]:
for layer in tqdm(custom_resnet_model.layers[:-1]):
	layer.trainable = False


100%|██████████| 178/178 [00:00<00:00, 44389.45it/s]


In [12]:
custom_resnet_model.layers[-1].trainable

custom_resnet_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
Noepochs=2
t=time.time()
hist = custom_resnet_model.fit(X_train, y_train, batch_size=32, epochs=Noepochs, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' % (t - time.time()))

Epoch 1/2
 119/1063 [==>...........................] - ETA: 28:16 - loss: 0.6721 - accuracy: 0.8041

In [13]:
custom_resnet_model.save("trained_model.h5")

In [ ]:
(loss, accuracy) = custom_resnet_model.evaluate(X_test, y_test, batch_size=10, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))